In [8]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Санкт-Петербург, улица Будапештская 95 корпус 1")
print((location.latitude, location.longitude))
coord = location.latitude, location.longitude

/home/caym/Programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


AttributeError: 'NoneType' object has no attribute 'latitude'

In [3]:
from geopy.geocoders import Bing
geolocator = Bing('Aue9DtrraMekDh2ugcmmFM3aZKsZYhYlFzceV8I70ouB6Ov_7JqtcEQclEjuI02V')
location = geolocator.geocode("Санкт-Петербург, Обводном Днепропетровской")
print((location.latitude, location.longitude))
coord = location.latitude, location.longitude

(59.93318176269531, 30.306039810180664)


In [7]:
from geopy.geocoders import Yandex
f = open('api_key.txt', 'r')
api = f.read()
geolocator = Yandex(api)
location = geolocator.geocode("Санкт-Петербург, Парашютной Долгоозёрной")
print((location.latitude, location.longitude))
coord = location.latitude, location.longitude

(60.020943, 30.264188)


In [5]:
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
import folium
geolocator = Nominatim()

map = folium.Map(location=[59.939095, 30.315868], zoom_start = 10)
marker_cluster = MarkerCluster().add_to(map)
for i in range(15):
    if 'ans' in test[i]:
        id = ind[i]
        location = geolocator.geocode(test[i]['ans'][0])
        folium.Marker(location=[location.latitude, location.longitude], popup = folium.Popup(f'<a href="https://vk.com/spb_today?w=wall-68471405_{id}"target="_blank">link to news in VK</a>'), icon=folium.Icon(color = 'red', icon='info-sign')).add_to(marker_cluster)
map.save("map1.html")